In [5]:
import pandas as pd
import requests
import time
import os

# --- Set file paths ---
pub_path = r"C:\Users\jawri\Data course\Capstone\CORDIS data\Projects\projectPublications.csv"
output_folder = r"C:\Users\jawri\Data course\Capstone\CORDIS data\Projects\alex"
os.makedirs(output_folder, exist_ok=True)

citations_path = os.path.join(output_folder, "project_citations.csv")
errors_path = os.path.join(output_folder, "citation_errors.csv")
email = "your email here"

# --- Load projectPublications.csv ---
publications = pd.read_csv(pub_path, sep=";", quoting=3, encoding="utf-8", on_bad_lines="skip")
publications.columns = publications.columns.str.strip().str.replace('"', '').str.lower()
publications['doi'] = publications['doi'].astype(str).str.strip().str.strip('"')

# --- Filter valid DOIs ---
valid_dois = publications['doi'].dropna().unique()
valid_dois = [d for d in valid_dois if d != '' and d != '0']

# --- Load existing citations if available ---
if os.path.exists(citations_path):
    existing = pd.read_csv(citations_path)
    already_done = set(existing['doi'].astype(str))
    print(f"✅ Loaded existing citations: {len(already_done)} DOIs already done.")
else:
    existing = pd.DataFrame(columns=['projectID', 'doi', 'cited_by_count'])
    already_done = set()
    print("ℹ️ No existing citation file found. Starting fresh.")

# --- Identify DOIs to process ---
new_dois = [doi for doi in valid_dois if doi not in already_done]
print(f"🔍 {len(new_dois)} new DOIs remaining.")

# --- Init containers ---
citation_data = []
failed_dois = []

# --- Start loop ---
for i, doi in enumerate(new_dois):
    url = f"https://api.openalex.org/works/doi:{doi}?mailto={email}"
    try:
        r = requests.get(url, timeout=10)
        if r.status_code == 200:
            cited_by = r.json().get("cited_by_count", None)
        else:
            cited_by = None
            failed_dois.append({'doi': doi, 'error': f"Status {r.status_code}"})
    except Exception as e:
        cited_by = None
        failed_dois.append({'doi': doi, 'error': str(e)})

    # Add all projectIDs using this DOI
    rows = publications[publications['doi'] == doi]
    for _, row in rows.iterrows():
        citation_data.append({
            "projectID": row['projectid'],
            "doi": doi,
            "cited_by_count": cited_by
        })

    # Save every 50
    if len(citation_data) % 50 == 0 or i == len(new_dois) - 1:
        temp_df = pd.DataFrame(citation_data)
        existing = pd.concat([existing, temp_df], ignore_index=True)
        existing.to_csv(citations_path, index=False)
        citation_data = []

        if failed_dois:
            err_df = pd.DataFrame(failed_dois)
            err_df.to_csv(errors_path, mode='a', index=False, header=not os.path.exists(errors_path))
            failed_dois = []

        print(f"💾 Saved progress at DOI {i+1} / {len(new_dois)}")

    time.sleep(0.1)

print("✅ Finished! All new citations saved.")


ℹ️ No existing citation file found. Starting fresh.
🔍 283149 new DOIs remaining.


C:\Users\jawri\AppData\Local\Temp\ipykernel_10412\4070029260.py:68: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  existing = pd.concat([existing, temp_df], ignore_index=True)


💾 Saved progress at DOI 42 / 283149
💾 Saved progress at DOI 86 / 283149
💾 Saved progress at DOI 129 / 283149
💾 Saved progress at DOI 204 / 283149
💾 Saved progress at DOI 250 / 283149
💾 Saved progress at DOI 296 / 283149
💾 Saved progress at DOI 341 / 283149
💾 Saved progress at DOI 390 / 283149
💾 Saved progress at DOI 438 / 283149
💾 Saved progress at DOI 481 / 283149
💾 Saved progress at DOI 526 / 283149
💾 Saved progress at DOI 573 / 283149
💾 Saved progress at DOI 620 / 283149
💾 Saved progress at DOI 668 / 283149
💾 Saved progress at DOI 716 / 283149
💾 Saved progress at DOI 762 / 283149
💾 Saved progress at DOI 802 / 283149
💾 Saved progress at DOI 849 / 283149
💾 Saved progress at DOI 897 / 283149
💾 Saved progress at DOI 945 / 283149
💾 Saved progress at DOI 986 / 283149
💾 Saved progress at DOI 1029 / 283149
💾 Saved progress at DOI 1070 / 283149
💾 Saved progress at DOI 1117 / 283149
💾 Saved progress at DOI 1160 / 283149
💾 Saved progress at DOI 1205 / 283149
💾 Saved progress at DOI 1250 / 2831

In [7]:
# Path to your citation file
citations_path = r"C:\Users\jawri\Data course\Capstone\CORDIS data\Projects\alex\project_citations.csv"

# Load it
citation_df = pd.read_csv(citations_path)

# Show first few rows
print("📄 Head of citation table:")
display(citation_df.head())

# Show summary stats
print("\n📊 Summary statistics:")
display(citation_df.describe())

# check data types and row count
print("\n🔎 Structure info:")
citation_df.info()

📄 Head of citation table:


,projectID,doi,cited_by_count
0,"""951475""",10.1016/j.eclinm.2022.101495,28.0
1,"""825664""",10.1002/dad2.12242,28.0
2,"""825664""",10.1001/jamaneurol.2021.3180,341.0
3,"""825664""",10.1021/jacs.3c02022,7.0
4,"""825546""",10.1016/j.mam.2024.101322,4.0



📊 Summary statistics:


,cited_by_count
count,205888.000000
mean,41.489237
std,150.800818
min,0.000000
25%,4.000000
50%,15.000000
75%,38.000000
max,19092.000000



🔎 Structure info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304433 entries, 0 to 304432
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   projectID       304433 non-null  object 
 1   doi             304420 non-null  object 
 2   cited_by_count  205888 non-null  float64
dtypes: float64(1), object(2)
memory usage: 7.0+ MB


In [11]:
errors_path = r"C:\Users\jawri\Data course\Capstone\CORDIS data\Projects\alex\citation_errors.csv"

# Load the failed DOI file
if os.path.exists(errors_path):
    errors_df = pd.read_csv(errors_path)
    print(f"❌ {len(errors_df)} failed DOIs found.")
    display(errors_df.head(10))  # Show first few
else:
    print("✅ No failed DOIs file found — all requests may have succeeded.")

❌ 91003 failed DOIs found.


,doi,error
0,10.48550/arxiv.2503.05876,Status 404
1,10.1103/physrevd.107.035013},Status 404
2,10.19283/lph2024.753,Status 404
3,10.33896/porient.2022.1–4.3,Status 404
4,10.25969/mediarep/19319,Status 404
5,10.13133/2532-6562/18849,Status 404
6,10.19272/202400601004,Status 404
7,10.69085/linc20251223,Status 404
8,10.52825/ocp.v5i.2342,Status 404
9,10.34813/ptr3.2022.10,Status 404


In [13]:
# Test a specific DOI manually
test_doi = "10.19283/lph2024.753"
url = f"https://api.openalex.org/works/doi:{test_doi}?mailto={email}"

r = requests.get(url, timeout=10)
print(f"Status code: {r.status_code}")

if r.status_code == 200:
    print("✅ OpenAlex returned data:")
    from pprint import pprint
    pprint(r.json())
else:
    print(f"❌ OpenAlex did not find the DOI. Response:")
    print(r.text)


Status code: 429
❌ OpenAlex did not find the DOI. Response:
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>429 Too Many Requests</title>
<h1>Too Many Requests</h1>
<p>100000 per 1 day</p>



In [15]:

# Load the failed DOIs
errors_path = r"C:\Users\jawri\Data course\Capstone\CORDIS data\Projects\alex\citation_errors.csv"
errors_df = pd.read_csv(errors_path)

# Define common "bad" trailing characters
bad_chars = set('}])"\'')

# Function to check for trailing bad characters
def has_bad_trailing(doi):
    return doi and doi[-1] in bad_chars

# Apply function
errors_df['has_bad_trailing'] = errors_df['doi'].astype(str).apply(has_bad_trailing)

# Count how many
count_bad = errors_df['has_bad_trailing'].sum()
total = len(errors_df)

print(f"❗ DOIs with trailing bad characters: {count_bad} / {total}")

# Optional: show a few examples
print("\n🔍 Example bad DOIs:")
print(errors_df[errors_df['has_bad_trailing']]['doi'].head(10).to_list())


❗ DOIs with trailing bad characters: 228 / 91003

🔍 Example bad DOIs:
['10.1103/physrevd.107.035013}', '10.3390/su15043008(registeringdoi)', '10.9770/jesi.2022.10.2(33)', '10.1007/978-981-13-0596-2_26++(springer.com)', '2020 21st IEEE International Conference on Mobile Data Management (MDM)', 'IEEE/CVF International Conference on Computer Vision (ICCV)', '17th Conference on Ph.D Research in Microelectronics and Electronics (PRIME)', '2022 IEEE International Symposium on Circuits and Systems (ISCAS)', '5th Planetary Data Workshop & Planetary Science Informatics & Analytics (Held virtually)', '7th IAA Planetary Defense Conference (held virtually)']


In [21]:
# Define characters you want to treat as bad at the end
bad_chars = set('}])"\'')

# Function to check for trailing junk
def has_bad_trailing(doi):
    try:
        return str(doi).strip()[-1] in bad_chars
    except:
        return False

# Apply safely
bad_trailing_flags = publications['doi'].apply(has_bad_trailing)

# Count how many DOIs in the original data had bad trailing characters
original_bad = bad_trailing_flags.sum()
print(f"❗ Original DOIs with bad trailing characters: {original_bad}")

# see examples
print("\n🔍 Examples:")
print(publications[bad_trailing_flags]['doi'].head(100).tolist())



❗ Original DOIs with bad trailing characters: 262

🔍 Examples:
['10.1103/physrevd.107.035013}', '10.52152/22.4.150-209(2024)', '10.3390/su15043008(registeringdoi)', '10.9770/jesi.2017.4.3S(8)', '10.9770/jesi.2022.10.2(33)', '10.9770/jesi.2015.3.1(2)', '10.9770/jesi.2017.4.3S(3)', '10.9770/jesi.2018.6.1(17)', '10.9770/ird.2019.1.2(7)', '10.9770/ird.2019.1.3(6)', '10.9770/ird.2019.1.3(5)', '10.9770/ird.2019.1.3(4)', '10.9770/ird.2019.1.1(2)', '10.9770/jesi.2017.5.1(4)', '10.9770/ird.2019.1.3(1)', '10.9770/ird.2019.1.1(1)', '10.9770/jesi.2017.4.4(6)', '10.9770/jesi.2017.5.2(3)', '10.9770/jesi.2016.3.4(1)', '10.9770/jesi.2016.4.1(7)', '10.1130/2021.2550(29)', '10.1007/978-981-13-0596-2_26++(springer.com)', '2020 21st IEEE International Conference on Mobile Data Management (MDM)', 'IEEE/CVF International Conference on Computer Vision (ICCV)', '17th Conference on Ph.D Research in Microelectronics and Electronics (PRIME)', '2022 IEEE International Symposium on Circuits and Systems (ISCAS)', '

In [25]:
# Ensure all DOIs are strings
publications['doi'] = publications['doi'].astype(str)

# Count DOIs that do NOT start with '10.'
invalid_count = (~publications['doi'].str.startswith("10.")).sum()

print(f"🚫 DOIs that do NOT start with '10.': {invalid_count}")



🚫 DOIs that do NOT start with '10.': 96521


In [27]:
# Make sure 'doi' and 'projectid' are treated correctly
publications['doi'] = publications['doi'].astype(str).str.strip()
publications['projectid'] = publications['projectid'].astype(str).str.strip()

# Group by DOI and count number of unique projectIDs per DOI
projects_per_doi = publications.groupby('doi')['projectid'].nunique()

# Summary statistics
print("📊 Summary of project counts per DOI:")
print(projects_per_doi.describe())

# Most shared DOIs (top 10)
print("\n🔝 DOIs linked to the most different projects:")
print(projects_per_doi.sort_values(ascending=False).head(10))


📊 Summary of project counts per DOI:
count    283145.000000
mean          1.108531
std          24.225760
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max       12890.000000
Name: projectid, dtype: float64

🔝 DOIs linked to the most different projects:
doi
                  12890
2022                 89
2021                 79
2020                 65
2018                 55
2023                 51
2019                 49
2017                 36
2016                 21
9781-510619524       16
Name: projectid, dtype: int64


In [29]:
# Load the citation results
citations_path = r"C:\Users\jawri\Data course\Capstone\CORDIS data\Projects\alex\project_citations.csv"
citations = pd.read_csv(citations_path)

# Ensure DOI column is string
citations['doi'] = citations['doi'].astype(str)

# Check how many start with "10."
valid_count = citations['doi'].str.startswith("10.").sum()
total_count = len(citations)

print(f"✅ DOIs starting with '10.': {valid_count} / {total_count}")
print(f"📉 Proportion: {valid_count / total_count:.2%}")

# show a few that don't start with '10.'
print("\n🔍 Examples that do NOT start with '10.':")
print(citations[~citations['doi'].str.startswith("10.")]['doi'].dropna().unique()[:10])


✅ DOIs starting with '10.': 295500 / 304433
📉 Proportion: 97.07%

🔍 Examples that do NOT start with '10.':
['20.500.11769/362880' '10230/46103' '101140/epjc/s10052-019-7140-6'
 '10278/3701560' '0.1002/ett.3070' '107346/-fei-XIV-03-16_06'
 'Krausmann, Fridolin' '1' '2' '217']


In [31]:
citations = pd.read_csv(citations_path)

dupes = citations.duplicated(subset=['projectID', 'doi']).sum()
print(f"🧾 Duplicate project–DOI pairs in citation file: {dupes}")


🧾 Duplicate project–DOI pairs in citation file: 3442


In [33]:
# Load the original publication file
pub = pd.read_csv(pub_path, sep=";", quoting=3, encoding="utf-8", on_bad_lines="skip")
pub.columns = pub.columns.str.strip().str.replace('"', '').str.lower()
pub['doi'] = pub['doi'].astype(str).str.strip()

# Count unique project–DOI pairs in the source
pub_unique_pairs = pub.drop_duplicates(subset=['projectid', 'doi'])
print(f"📦 Unique project–DOI pairs in projectPublications.csv: {len(pub_unique_pairs)}")


📦 Unique project–DOI pairs in projectPublications.csv: 313884


In [35]:
pub = pd.read_csv(pub_path, sep=";", quoting=3, encoding="utf-8", on_bad_lines="skip")
pub.columns = pub.columns.str.strip().str.replace('"', '').str.lower()
pub['doi'] = pub['doi'].astype(str).str.strip()

print(f"📄 Total rows in projectPublications.csv: {len(pub)}")
print(f"🧾 Unique project–DOI pairs: {pub.drop_duplicates(subset=['projectid', 'doi']).shape[0]}")


📄 Total rows in projectPublications.csv: 392021
🧾 Unique project–DOI pairs: 313884


In [37]:
citations = pd.read_csv(citations_path)

# Clean up just in case
citations['doi'] = citations['doi'].astype(str).str.strip()

# Add length column
citations['doi_len'] = citations['doi'].str.len()

# Find shortest length and example(s)
min_len = citations['doi_len'].min()
shortest_dois = citations[citations['doi_len'] == min_len]['doi'].unique()

print(f"📏 Shortest DOI length: {min_len}")
print("🔍 Example(s):", shortest_dois[:5])


📏 Shortest DOI length: 1
🔍 Example(s): ['1' '2' '9' '5' '3']


In [39]:


# Keep only rows where citation count was successfully retrieved
valid_citations = citations[citations['cited_by_count'].notna()]

print(f"✅ Valid citation rows: {len(valid_citations)} / {len(citations)}")
print(valid_citations.head())



✅ Valid citation rows: 205888 / 304433
  projectID                           doi  cited_by_count  doi_len
0  "951475"  10.1016/j.eclinm.2022.101495            28.0       28
1  "825664"            10.1002/dad2.12242            28.0       18
2  "825664"  10.1001/jamaneurol.2021.3180           341.0       28
3  "825664"          10.1021/jacs.3c02022             7.0       20
4  "825546"     10.1016/j.mam.2024.101322             4.0       25


In [45]:
# Test a specific DOI manually
test_doi = "10.1103/physrevd.107.035013"
url = f"https://api.openalex.org/works/doi:{test_doi}?mailto={email}"

r = requests.get(url, timeout=10)
print(f"Status code: {r.status_code}")

if r.status_code == 200:
    data = r.json()
    print("✅ OpenAlex returned data:")
    print(f"📊 Citation count: {data.get('cited_by_count', 'Not available')}")
else:
    print(f"❌ OpenAlex did not find the DOI. Response:")
    print(r.text)


Status code: 200
✅ OpenAlex returned data:
📊 Citation count: 1


In [49]:
# --- Load publications to map DOIs to projectIDs ---
publications = pd.read_csv(pub_path, sep=";", quoting=3, encoding="utf-8", on_bad_lines="skip")
publications.columns = publications.columns.str.strip().str.replace('"', '').str.lower()
publications['doi'] = publications['doi'].astype(str).str.strip().str.strip('"')

# --- Load citation_errors.csv ---
errors = pd.read_csv(errors_path)
errors['doi'] = errors['doi'].astype(str).str.strip().str.strip('"').str.strip('}])\'"+')

# Drop short or clearly junk DOIs
pre_filter_count = len(errors)
errors = errors[errors['doi'].str.len() > 5]
print(f"🧼 Dropped {pre_filter_count - len(errors)} DOIs with length <= 5")

# Ensure 'status' column exists
if 'status' not in errors.columns:
    errors['status'] = 'pending'

# Only retry ones not marked as success
errors_to_retry = errors[errors['status'] != 'success'].copy()
retry_dois = errors_to_retry['doi'].drop_duplicates().tolist()
print(f"📦 Total DOIs to retry after filtering: {len(retry_dois)}")

# --- Load existing citations ---
existing = pd.read_csv(citations_path)
already_done_pairs = set(zip(existing['projectID'].astype(str), existing['doi'].astype(str)))
print(f"✅ Loaded existing citations: {len(already_done_pairs)} project–DOI pairs already done.")

# --- Init containers ---
citation_data = []
failed_updates = []

# --- Retry loop ---
for i, doi in enumerate(retry_dois):
    url = f"https://api.openalex.org/works/doi:{doi}?mailto={email}"
    try:
        r = requests.get(url, timeout=10)
        status = r.status_code
        if status == 200:
            cited_by = r.json().get("cited_by_count", None)
        elif status == 429:
            print("🚫 RATE LIMIT REACHED: Too many requests — exiting loop.")
            break
        else:
            raise Exception(f"Status {status}")
    except Exception as e:
        errors.loc[errors['doi'] == doi, 'status'] = 'failed'
        if i % 10 == 0:
            print(f"❌ DOI {i+1}/{len(retry_dois)} failed: {doi} — {e}")
        continue

    # Lookup all projectIDs using this DOI
    rows = publications[publications['doi'] == doi]
    if rows.empty:
        errors.loc[errors['doi'] == doi, 'status'] = 'failed'
        if i % 10 == 0:
            print(f"⚠️ DOI {i+1}: {doi} returned data but matched 0 projectIDs")
        continue

    added_any = False
    for _, row in rows.iterrows():
        pair = (str(row['projectid']), doi)
        if pair not in already_done_pairs:
            citation_data.append({
                "projectID": row['projectid'],
                "doi": doi,
                "cited_by_count": cited_by
            })
            already_done_pairs.add(pair)
            added_any = True

    if added_any:
        errors.loc[errors['doi'] == doi, 'status'] = 'success'
    else:
        errors.loc[errors['doi'] == doi, 'status'] = 'failed'

    # Print status every 10
    if i % 10 == 0:
        print(f"🔁 {i+1}/{len(retry_dois)} — DOI: {doi} → Citations: {cited_by} | Projects linked: {len(rows)}")

    # Save every 50
    if len(citation_data) % 50 == 0 or i == len(retry_dois) - 1:
        if citation_data:
            temp_df = pd.DataFrame(citation_data)
            existing = pd.concat([existing, temp_df], ignore_index=True)
            existing.to_csv(citations_path, index=False)
            citation_data = []
            print(f"💾 Saved progress at DOI {i+1} / {len(retry_dois)}")

        errors.to_csv(errors_path, index=False)

    time.sleep(0.1)

print("✅ Retry loop complete.")



🧼 Dropped 0 DOIs with length <= 5
📦 Total DOIs to retry after filtering: 4851
✅ Loaded existing citations: 300991 project–DOI pairs already done.
❌ DOI 1/4851 failed: 10.48550/arxiv.2503.05876 — Status 404
❌ DOI 11/4851 failed: 10.48550/arxiv.2504.01232 — Status 404
❌ DOI 21/4851 failed: 10.15199/42.2023.2.2 — Status 404
❌ DOI 31/4851 failed: 10.1049/stg2.12142ss — Status 404
❌ DOI 41/4851 failed: 10.19282/ac.32.1.2021.22 — Status 404
❌ DOI 51/4851 failed: 10.19229/2464-9309/11202022 — Status 404
❌ DOI 61/4851 failed: 10.57660/dpceonline.2023.1961 — Status 404
❌ DOI 71/4851 failed: 10.1073/pnas.210603111 — Status 404
❌ DOI 81/4851 failed: 10.5281/zenodo.10512965 — Status 404
❌ DOI 91/4851 failed: 10.1177/0170840623116200 — Status 404
❌ DOI 101/4851 failed: 10.1021/jacs.1c07497. — Status 404
❌ DOI 111/4851 failed: 10.5281/zenodo.4009797 — Status 404
❌ DOI 121/4851 failed: 10.1393/ncc/i2023-23145-3 — Status 404
❌ DOI 131/4851 failed: 10.1177/1079063223119008 — Status 404
❌ DOI 141/4851 f

In [55]:
# Load the updated citations file
citations_path = r"C:\Users\jawri\Data course\Capstone\CORDIS data\Projects\alex\project_citations.csv"
citations_df = pd.read_csv(citations_path)

print("📊 Total rows in citations (raw):", len(citations_df))
print("🔍 Unique DOIs:", citations_df['doi'].nunique())
print("🧮 Unique projectIDs:", citations_df['projectID'].nunique())

# 🧼 Drop duplicate (projectID, doi) pairs to avoid double counting
citations_df = citations_df.drop_duplicates(subset=['projectID', 'doi'])
print("✅ Rows after removing duplicates:", len(citations_df))

# Total citation count per projectID
project_citation_totals = citations_df.groupby('projectID')['cited_by_count'].sum().reset_index()
project_citation_totals = project_citation_totals.sort_values(by='cited_by_count', ascending=False)

# Show top 10
print("\n🏆 Top 10 projects by total citations:")
print(project_citation_totals.head(10))



📊 Total rows in citations (raw): 304433
🔍 Unique DOIs: 283146
🧮 Unique projectIDs: 24984
✅ Rows after removing duplicates: 300991

🏆 Top 10 projects by total citations:
      projectID  cited_by_count
4909   "633053"        135748.0
9755   "696656"        100070.0
9352   "692145"         80741.0
15445  "785219"         62017.0
5590   "641816"         45849.0
21463  "871029"         36820.0
7575   "665667"         34974.0
15484  "785907"         33018.0
6257   "647839"         29827.0
12724  "743080"         28238.0


In [57]:
citations_df[citations_df['projectID'] == '"633053"']['doi'].nunique()


7332

In [59]:
citations_df[citations_df['projectID'] == '"633053"'].sort_values(by='cited_by_count', ascending=False).head(10)


,projectID,doi,cited_by_count
50049,"""633053""",10.1016/j.nds.2019.01.002,664.0
45764,"""633053""",10.1126/sciadv.aav2002,515.0
49709,"""633053""",10.1016/j.jnucmat.2018.10.027,486.0
46793,"""633053""",10.1103/PhysRevLett.116.135504,440.0
50334,"""633053""",10.1016/j.jnucmat.2020.152600,423.0
50098,"""633053""",10.1103/revmodphys.88.015008,405.0
45315,"""633053""",10.1016/j.jnucmat.2014.10.075,340.0
47084,"""633053""",10.1098/rsta.2017.0432,319.0
47446,"""633053""",10.1088/0029-5515/54/3/033007,306.0
51398,"""633053""",10.1038/s41467-018-03415-5,305.0


In [61]:
# Count how many DOIs overlap between "633053" and other projects
dois_633053 = set(citations_df[citations_df['projectID'] == '"633053"']['doi'])
other_projects = citations_df[citations_df['projectID'] != '"633053"']
shared_dois = other_projects[other_projects['doi'].isin(dois_633053)]

# How many of those DOIs are shared?
print(f"Shared DOIs with other projects: {shared_dois['doi'].nunique()}")


Shared DOIs with other projects: 144


In [73]:
# Load the project data
proj_path = r"C:\Users\jawri\Data course\Capstone\CORDIS data\Projects\project.csv"
df = pd.read_csv(proj_path, sep=";", encoding="utf-8", quoting=3, on_bad_lines="skip")
df.columns = df.columns.str.strip().str.replace('"', '').str.lower()

# Force id column to string and strip extra spaces or quotes
df['id'] = df['id'].astype(str).str.strip().str.strip('"')

# Check what the first few IDs look like
print("🔍 Sample project IDs:")
print(df['id'].head(10).tolist())

# Try again to filter the row
target_id = '633053'
row = df[df['id'] == target_id]
print(f"\n📄 Project row for ID {target_id}:")
print(row)


🔍 Sample project IDs:
['672641', '750248', '855867', '778157', '821431', '831756', '699794', '809304', '101003436', '846028']

📄 Project row for ID 633053:
Empty DataFrame
Columns: [id, acronym, status, title, startdate, enddate, totalcost, ecmaxcontribution, legalbasis, topics, ecsignaturedate, frameworkprogramme, mastercall, subcall, fundingscheme, nature, objective, contentupdatedate, rcn, grantdoi]
Index: []


In [77]:
# Compare unique project IDs
unique_citation_ids = citations_df['projectID'].astype(str).str.strip().unique()
unique_project_ids = projects_df['id'].astype(str).str.strip().unique()

matched_ids = set(unique_citation_ids).intersection(set(unique_project_ids))
num_matched = len(matched_ids)
num_total = len(unique_citation_ids)
num_missing = num_total - num_matched

print(f"✅ Unique project IDs matched: {num_matched} / {num_total}")
print(f"❌ Unique project IDs missing from projects file: {num_missing}")



✅ Unique project IDs matched: 16633 / 24980
❌ Unique project IDs missing from projects file: 8347


In [79]:
existing.shape
existing.head()


,projectID,doi,cited_by_count
0,"""951475""",10.1016/j.eclinm.2022.101495,28.0
1,"""825664""",10.1002/dad2.12242,28.0
2,"""825664""",10.1001/jamaneurol.2021.3180,341.0
3,"""825664""",10.1021/jacs.3c02022,7.0
4,"""825546""",10.1016/j.mam.2024.101322,4.0


In [81]:
len(existing)


304433

In [83]:
existing.to_csv("recovered_project_citations.csv", index=False)


In [93]:
import csv
# Load the projects file
project_path = r"C:\Users\jawri\Data course\Capstone\CORDIS data\Projects\project.csv"
project_df = pd.read_csv(
    project_path,
    sep=';',
    quoting=csv.QUOTE_ALL,
    encoding='utf-8',
    on_bad_lines='skip'
)

# Load the recovered citations
citations = pd.read_csv("recovered_project_citations.csv")
# Remove quotes from projectID in citations
citations['projectID'] = citations['projectID'].str.strip('"')

# Convert projectID to string in both to ensure consistent comparison
project_df['projectID'] = project_df['id'].astype(str)
citations['projectID'] = citations['projectID'].astype(str)

# Compare projectIDs
print(f"Unique projectIDs in project_df: {project_df['id'].nunique()}")
print(f"Unique projectIDs in citations: {citations['projectID'].nunique()}")

# Check how many citation projectIDs match those in project_df
matched = citations['projectID'].isin(project_df['id'])
print(f"Matched projectIDs: {matched.sum()}")
print(f"Unmatched projectIDs: {(~matched).sum()}")


Unique projectIDs in project_df: 35105
Unique projectIDs in citations: 24973
Matched projectIDs: 0
Unmatched projectIDs: 304433


In [95]:
print("Projects:", project_df['id'].astype(str).unique()[:5])
print("Citations:", citations['projectID'].unique()[:5])


Projects: ['672641' '750248' '855867' '778157' '798830']
Citations: ['951475' '825664' '825546' '956454' '956439']


In [97]:
import os
print(os.getcwd())


C:\Users\jawri
